# エージェントが受け取るシステムプロンプトを確認する方法

In [1]:
import os
[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'TRUE'

## BaseLlmFlow クラスの _run_one_step_async() メソッドを書き換える

_run_one_step_async() メソッドは、LLM の API 呼び出しを 1 回実行します。

この中で、LLM に渡す contents や config が用意されるので、その内容をプリントします。

In [2]:
import pprint
from google.adk.agents.invocation_context import InvocationContext
from typing import AsyncGenerator
from google.adk.events.event import Event
from google.adk.models.llm_request import LlmRequest

async def _run_one_step_async2(
      self,
      invocation_context: InvocationContext,
  ) -> AsyncGenerator[Event, None]:
    """One step means one LLM call."""
    llm_request = LlmRequest()

    # Preprocess before calling the LLM.
    async for event in self._preprocess_async(invocation_context, llm_request):
        yield event
    if invocation_context.end_invocation:
        return

    # Calls the LLM.
    model_response_event = Event(
        id=Event.new_id(),
        invocation_id=invocation_context.invocation_id,
        author=invocation_context.agent.name,
        branch=invocation_context.branch,
    )

    ## DEBUG output
    if DEBUG:
        print('## Prompt contents ##')
        pprint.pp(llm_request.contents)
        print('----')
        print('## System instruction ##')
        print(llm_request.config.system_instruction)
        print('----')
    ####

    async for llm_response in self._call_llm_async(
        invocation_context, llm_request, model_response_event
    ):
        # Postprocess after calling the LLM.
        async for event in self._postprocess_async(
            invocation_context, llm_request, llm_response, model_response_event
        ):
            yield event

from google.adk.flows.llm_flows.base_llm_flow import BaseLlmFlow
BaseLlmFlow._run_one_step_async = _run_one_step_async2

## 実行例

In [3]:
import uuid
from google.genai.types import (
    HttpOptions, GenerateContentConfig,
    Part, UserContent, ModelContent
)
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.agents import LlmAgent

ダミーツールを持った天気予報エージェントを定義します。

In [4]:
# Tool
def get_weather(city: str) -> str:
    """Retrieves weather information for the given city.

    Args:
        city: The name of the city in English for which to retrieve weather information.

    Returns:
        A string containing the weather information for the specified city,
        or a message indicating that the weather information was not found.
    """
    cities = {
        'chicago': {'temperature': 25, 'condition': 'sunny', 'sky': 'clear'},
        'toronto': {'temperature': 30, 'condition': 'partly cloudy', 'sky': 'overcast'},
        'chennai': {'temperature': 15, 'condition': 'rainy', 'sky': 'cloudy'},
    }

    city_lower = city.lower()
    if city_lower in cities:
        weather_data = cities[city_lower]
        return f"Weather in {city} is {weather_data['temperature']} degrees Celsius, {weather_data['condition']} with a {weather_data['sky']} sky."
    else:
        return f"Weather information for {city} not found."

# Agent
weather_agent = LlmAgent(
    model='gemini-2.0-flash-001',
    name='weather_agent',
    instruction="""
    You are a Weather Information Agent.
    Your task is to provide weather information for a given city.
    Use the `get_weather` tool to retrieve the weather information.
    """,
    description="""You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task.""",
    tools=[get_weather],
)

In [5]:
class LocalApp:
    def __init__(self, agent, user_id='default_user'):
        self._agent = agent
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._agent.name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = self._runner.session_service.create_session(
            app_name=self._agent.name,
            user_id=self._user_id,
            state={},
            session_id=uuid.uuid4().hex,
        )
        
    async def stream(self, query):
        content = UserContent(parts=[Part.from_text(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print('## Event (LLM output or function result) ##')
                print(event)
                print('----')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

次の様に、system instruction には、`instruction` オプションで指定した内容に加えて、ツールの情報などが自動で追加されています。

また、contents（ユーザープロンプト）には、過去のやり取りがすべて含められています。

In [6]:
client = LocalApp(weather_agent)

DEBUG = True
query = '''
What's the weather in Cicago?
'''
_ = await client.stream(query)

## Prompt contents ##
[UserContent(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="\nWhat's the weather in Cicago?\n")], role='user')]
----
## System instruction ##

    You are a Weather Information Agent.
    Your task is to provide weather information for a given city.
    Use the `get_weather` tool to retrieve the weather information.
    

You are an agent. Your internal name is "weather_agent".

 The description about you is "You are an agent who can fetch weather information for a city.
    You have access to the `get_weather` tool to accomplish this task."
----


## Event (LLM output or function result) ##
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id='adk-76ec9815-6faf-41e1-967f-e97406594de6', args={'city': 'Chicago'}, name='get_weather'), function_response=None, inline_data=None, text=None)], role='model') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None invocation_id='e-72dd1500-2c60-419d-add4-d897495200ec' author='weather_agent' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=set() branch=None id='sQn5YoOe' timestamp=1746519775.065599
----
## Event (LLM output or function result) ##
content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=Functi